In [1]:
import pandas as pd
import numpy as np


import sys

sys.path.append("../src/utils")

import parsing as utpa

In [2]:
icd_raw_dir = "/data6/deepro/ukb_bmi/0_data_preparation_and_download/icd_codes/data/icd_raw"
icd_codes_file = "/data6/deepro/ukb_bmi/0_data_preparation_and_download/icd_codes/data/icd_tree/coding19.tsv"

combo_files = [
    "/data6/deepro/ukb_bmi/3_characterization/data/combo_info/british/discovery_combo2.csv",
    "/data6/deepro/ukb_bmi/3_characterization/data/combo_info/british/discovery_combo3.csv"
]
cohort_file = "/data6/deepro/ukb_bmi/0_data_preparation_and_download/phenotype/data/bmi_processed/british/train_cohort_bmi.csv.gz"
hes_info_file = "/data6/deepro/ukb_bmi/0_data_preparation_and_download/icd_codes/data/hes_info/hes_info.csv.gz"

In [3]:
def create_phenotype_samples_df(phenotype_df, combo_samples):
    phenotype_df["carrier"] = phenotype_df.sample_names.isin(combo_samples)
    return phenotype_df

In [4]:
cohort_df = pd.read_csv(cohort_file, usecols=["sample_names",  "bmi", "bmi_prs", "bmi_residuals"], dtype={"sample_names": str, "bmi": float, "bmi_prs": float, "bmi_residuals":float})
cohort_df["sample_names"] = cohort_df.sample_names.astype(str)
combo_genes, combo_samples = utpa.get_combo_info_from_files(combo_files)
phenotype_samples_df = create_phenotype_samples_df(cohort_df, combo_samples)
icd_samples_df = utpa.create_icd_samples_file(icd_raw_dir)
icd_codes_df = pd.read_csv(icd_codes_file, usecols=["coding", "meaning", "node_id", "parent_id"], sep="\t")
icd_codes_df["coding"] = icd_codes_df.coding.str.replace(" ", "")
pheno_tree, root_pheno, c2nodeid_dict = utpa.create_tree(icd_codes_df, icd_samples_df)
hes_info_df = pd.read_csv(hes_info_file, dtype={"sample_names": str, "hes_info": float})
all_icd_samples = set(hes_info_df.loc[hes_info_df.hes_info>0, "sample_names"].values)
print(f"Samples in cohort {len(phenotype_samples_df)}")
phenotype_samples_df = phenotype_samples_df.loc[phenotype_samples_df.sample_names.isin(all_icd_samples)]
print(f"Samples with icd {len(phenotype_samples_df)}")
print(f"Samples with combo {len(combo_samples)}")
combo_samples = combo_samples.intersection(all_icd_samples)
print(f"Samples with combo and icd {len(combo_samples)}")

Samples in cohort 427639
Samples with icd 383253
Samples with combo 10193
Samples with combo and icd 9269


In [5]:
len(all_icd_samples)

449090